In [10]:
# 기본 패키지
import re
import time
import pickle
import pandas as pd
from tqdm import tqdm
from tqdm import trange
import warnings
warnings.filterwarnings('ignore')

# 크롤링 패키지
import requests
from selenium import webdriver
import myslack

In [5]:
# 0. 페이지 한번 맨아래로 내리기
def scroll_bottom():
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

# 1. 입력한 위치로 설정하기
def set_location(location):
    print(location+'으로 위치 설정 하는중...')
    driver.find_element_by_css_selector('#search > div > form > input').click()
    driver.find_element_by_css_selector('#button_search_address > button.btn-search-location-cancel.btn-search-location.btn.btn-default > span').click()
    driver.find_element_by_css_selector('#search > div > form > input').send_keys(location)
    driver.find_element_by_css_selector('#button_search_address > button.btn.btn-default.ico-pick').click()
    time.sleep(2)
    print(location+'으로 위치 설정 완료!')
    
# 2-1. 카테고리 페이지 Element Number Dictionary 정의
food_dict = { '프랜차이즈':3, '치킨':4, '피자&양식':5, '중국집':6,
              '한식':7, '일식&돈까스':8, '족발&보쌈':9,
              '야식':10, '분식':11, '카페&디저트':12 }

# 2-2. 카테고리 페이지로 넘어가기
def go_to_category(category):
    print(category+' 카테고리 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(food_dict.get(category))).click()
    time.sleep(4)
    print(category+' 카테고리 페이지 로드 완료!')    

# 3. 카테고리(음식점 리스트) 페이지 모두 펼치기
def stretch_list_page():
    restaurant_count = int(driver.find_element_by_css_selector('#restaurant_count').text)
    scroll_count = int((restaurant_count/20))
    print('모든 음식점 리스트 불러오기 시작...')
    for _ in trange(scroll_count):
        try:
            scroll_bottom()
            time.sleep(5)
        except Exception as e:
            pass
    scroll_bottom()
    time.sleep(5)
    print('모든 음식점 리스트 불러오기 완료!')
    
# 4. 해당 카테고리 음식점 리스트 리턴
def get_restaurant_list():
    restaurant_list=[]
    restaurants = driver.find_elements_by_css_selector('#content > div > div.restaurant-list > div.col-sm-6.contract')
    for restaurant in restaurants:
        restaurant_list.append(restaurant.find_element_by_css_selector('div > table > tbody > tr > td:nth-child(2) > div > div.restaurant-name.ng-binding').text)
    return list(set(restaurant_list))

# 5. 검색창에 음식점 검색하기
def search_restaurant(restaurant_name):
    try:
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[1]/a').click()
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[13]/form/div/input').send_keys(restaurant_name)    
        driver.find_element_by_xpath('//*[@id="category_search_button"]').click()
    except Exception as e:
        print('search_restaurant 에러')
    time.sleep(5)

# 6. 검색한 음식점 클릭
def go_to_restaurant():
    try:
        driver.find_element_by_xpath('//*[@id="content"]/div/div[4]/div[2]/div').click()
    except Exception as e:
        print('go_to_restaurant 에러')
    time.sleep(5)

# 7. 해당 음식점의 리뷰 페이지로 넘어가기
def go_to_review():
    print('리뷰 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(2)
    print('리뷰 페이지 로드 완료!')
    
# 8. 리뷰 더보기 클릭하기 
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()
    time.sleep(2)
    
# 9. 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in trange(click_count):
        try:
            scroll_bottom()
            click_more_review()
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 리뷰 불러오기 완료!')
        
# 10. 해당 음식점의 모든 리뷰 객체 리턴
def get_all_review_elements():
    reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    return reviews
        
# 11. 페이지 뒤로 가기 (한 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감)
def go_back_page():
    print('페이지 돌아가기중...')
    driver.execute_script("window.history.go(-1)")
    time.sleep(5)
    print('페이지 돌아가기 완료!'+'\n')
    
# 12. 크롤링과 결과 데이터를 pickle 파일로 저장
def save_pickle(location, category, yogiyo_df):
    pickle.dump(yogiyo_df, open('./data/{}_{}_df.pkl'.format(location, category),'wb'))
    print('{} {} pikcle save complete!'.format(location, category))

In [6]:
# 13. 크롤링 메인 함수
def yogiyo_crawling(location, category):
    df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                                   'Total','Taste','Quantity','Delivery','Date'])
    
    try:
        set_location(location) # 검색할 지역 설정
        go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감

        print('Start [ {} - {} ] Crawling...'.format(location, category))
        
        stretch_list_page() # 카테고리(음식점 리스트) 페이지 모두 펼치기
        restaurant_list = get_restaurant_list() # 해당 카테고리 음식점 리스트 받아오기
        
        for restaurant_name in restaurant_list:
            try:
                print('********** '+restaurant_name+' ( '+str(restaurant_list.index(restaurant_name)+1)
                      +'/'+str(len(restaurant_list))+' 번째) **********')
                
                search_restaurant(restaurant_name) # 음식점을 순서대로 검색
                go_to_restaurant() # 검색한 음식점 클릭             
                go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
                stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴

                for review in tqdm(get_all_review_elements()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
                    try:
                        df.loc[len(df)] = { 
                            'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                            'UserID':review.find_element_by_css_selector('span.review-id.ng-binding').text,
                            'Menu':review.find_element_by_css_selector('div.order-items.default.ng-binding').text,
                            'Review':review.find_element_by_css_selector('p').text,
                            'Total':str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                            'Taste':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                            'Quantity':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                            'Delivery':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text,
                            'Date':review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text,
                        }
                    except Exception as e:
                        print('리뷰 페이지 에러')
                        print(e)
                        pass
                    
            except Exception as e:
                print('*** '+restaurant_name+' *** 음식점 페이지 에러')
                go_back_page()
                print(e)
                pass
            
            print('음식점 리스트 페이지로 돌아가는중...')
            go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
            
    except Exception as e:
        print('음식점 리스트 페이지 에러')
        print(e)
        pass
        
    print('End of [ {} - {} ] Crawling!'.format(location, category))
    save_pickle(location, category, df) # 해당 음식점 리뷰 데이터를 pickle 파일로 저장함
    myslack.send_slack('{} {} crawling finish!!!'.format(location, category))
    
    return df 

In [17]:
# 14. 강남구 모든 관할구역(동) 크롤링 실행 함수
def start_gangnamgu_crawling(category):
    
    gangnamgu = ['역삼동','개포동','청담동','삼성동','대치동','신사동','논현동',
                 '압구정동','세곡동','자곡동','율현동','일원동','수서동','도곡동']
    
    for dong in gangnamgu:
        try:
            yogiyo = yogiyo_crawling('강남구 {}'.format(dong), category)
            print(dong+' - '+category+', shape: '+str(yogiyo.shape)+'\n')
        except Exception as e:
            print('***** '+dong+' 에러 발생 *****')
            print(e)
            pass

---

In [8]:
# Chrome webdriver - 요기요 메인 페이지 실행
driver = webdriver.Chrome() 
url = 'https://www.yogiyo.co.kr/'
driver.get(url)

In [18]:
%time start_gangnamgu_crawling('치킨')

강남구 논현동으로 위치 설정 하는중...
강남구 논현동으로 위치 설정 완료!
치킨 카테고리 페이지 로드중...


  0%|          | 0/3 [00:00<?, ?it/s]

치킨 카테고리 페이지 로드 완료!
Start [ 강남구 논현동 - 치킨 ] Crawling...
모든 음식점 리스트 불러오기 시작...


100%|██████████| 3/3 [00:15<00:00,  5.01s/it]


모든 음식점 리스트 불러오기 완료!
********** 파파이스-신논현역점 ( 1/19 번째) **********
리뷰 페이지 로드중...


...

리뷰 페이지 로드 완료!
모든 리뷰 불러오기 시작...


  0%|          | 0/416 [00:00<?, ?it/s]

모든 리뷰 불러오기 완료!


100%|██████████| 416/416 [02:14<00:00,  3.10it/s]


음식점 리스트 페이지로 돌아가는중...
페이지 돌아가기중...
페이지 돌아가기 완료!

End of [ 강남구 논현동 - 치킨 ] Crawling!
강남구 논현동 치킨 pikcle save complete!
<Response [200]>
논현동 - 치킨, shape: (9735, 9)

CPU times: user 2min 5s, sys: 8.19 s, total: 2min 13s
Wall time: 3h 54min 22s


In [22]:
df = pickle.load(open('./data/강남구 논현동_치킨_df.pkl','rb'))

In [23]:
df.tail()

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
9730,델리치킨,rl**님,,완전단골임ㅋ,5,,,,2013년 10월 10일
9731,델리치킨,rl**님,,완전맛나양념치킨짱조아^^,5,,,,2013년 10월 4일
9732,델리치킨,78**님,,배달시간보다 일찍왓구 친절하구 맛있네요!,5,,,,2013년 6월 23일
9733,델리치킨,23**님,,양이 많아서 좋아요,5,,,,2013년 1월 31일
9734,델리치킨,12**님,,왜캐 안와요?;;1시간이 다되가네 ..;;;;;어휴땀나;;,2,,,,2012년 11월 21일


In [24]:
df['Restaurant'].value_counts()

셰플리-셰프의건강한한끼(강남)    2262
후아바베큐-논현직영점         1400
강정이기가막혀-역삼점         1218
델리치킨                 832
키즈앤맘                 705
페리카나-논현점             655
롯데리아-학동역점            645
파파이스-신논현역점           523
또래오래-강남논현점           510
가성비닭                 221
양동시장통닭-언주역본점         207
KFC-학동역점             185
502세컨즈카페-강남점         135
치킨더홈-논현점             118
신불닭(논현동)              59
델리치킨오븐구이&강정           48
피치슈퍼마켓                12
Name: Restaurant, dtype: int64

In [25]:
# Chrome webdriver 종료
driver.close() 
driver.quit() 

---